In [7]:
import os
import numpy as np
import pandas as pd

In [8]:
def terma_detect_peaks(input_dir, output_dir, w_cycle=55, w_evt=9, beta=0.095):
    """
    Processes multiple PPG files, detects peaks, and saves the results into CSV files.
    
    Parameters:
    - input_dir: Path to the directory containing input PPG files.
    - output_dir: Path to the directory where the output files will be saved.
    - w_cycle: Window size for calculating the cycle moving average.
    - w_evt: Window size for calculating the event moving average.
    - beta: Scaling factor for the threshold.
    """
    
    # Loop over all files in the input directory
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.csv'):  # Process only CSV files
            input_file = os.path.join(input_dir, file_name)
            print(f"Processing file: {file_name}")
            
            # Read the PPG data from the file
            ppg_data = pd.read_csv(input_file, header=None)
            signal = np.asarray(ppg_data.iloc[:, 0], dtype=np.float64)  # Assume the PPG data is in the first column
            
            # Shift the signal to eliminate negative values
            min_signal = np.min(signal)
            if min_signal < 0:
                signal = signal - min_signal  # Shift signal so the minimum value becomes 0 or greater
            
            # Enhance the signal (square the values)
            signal = signal * signal
            
            # Initialize arrays for moving averages and thresholds
            ma_cycle = np.zeros_like(signal, dtype=np.float64)
            ma_evt = np.zeros_like(signal, dtype=np.float64)
            mean_signal = np.mean(signal)
            mean = np.full(len(signal), mean_signal)
            threshold_1 = np.zeros_like(signal, dtype=np.float64)
            block_of_interest = np.zeros_like(signal, dtype=np.float64)
            peak_index_arr = []
            
            # Calculate the Event Duration Moving Average
            for i in range((w_evt - 1) // 2, len(signal) - ((w_evt - 1) // 2)):
                ma_evt[i] = np.mean(signal[i - (w_evt - 1) // 2 : i + (w_evt - 1) // 2 + 1])
            
            # Calculate the Event Cycle Moving Average
            for i in range((w_cycle - 1) // 2, len(signal) - ((w_cycle - 1) // 2)):
                ma_cycle[i] = np.mean(signal[i - (w_cycle - 1) // 2 : i + (w_cycle - 1) // 2 + 1])
            
            # Calculate the Threshold for Block of Interest
            for i in range(len(signal)):
                threshold_1[i] = ma_cycle[i] + beta * mean[i]
            
            # Generate the Block of Interest
            for i in range(len(signal)):
                if (ma_evt[i] > threshold_1[i]) and (ma_cycle[i] != 0):
                    block_of_interest[i] = 1
                else:
                    block_of_interest[i] = 0
            
            # Peak detector logic
            start_block = 0
            stop_block = 0
            peak = 0
            peak_index = 0
            block_num = 0
            
            for i in range(len(block_of_interest) - 1):
                if (block_of_interest[i + 1] - block_of_interest[i]) == 1:
                    start_block = i
                if (block_of_interest[i] - block_of_interest[i + 1]) == 1:
                    stop_block = i
                    if (stop_block - start_block) >= w_evt:
                        peak = signal[start_block]
                        for j in range(start_block, stop_block + 1):
                            if signal[j] > peak:
                                peak = signal[j]
                                peak_index = j
                        peak_index_arr.append(peak_index)
                        block_num += 1
            
            # Save the peak indices to a CSV file
            output_file = os.path.join(output_dir, f"{file_name.replace('.csv', '')}_labeled_peaks.csv")
            pd.DataFrame(peak_index_arr).to_csv(output_file, header=False, index=False)
            print(f"Peaks saved to: {output_file}")


In [9]:
input_folder = 'I:/dilated_cnn_peak_detection_model_data/test/test_data/data'
output_folder = 'I:/dilated_cnn_peak_detection_model_data/test/test_data/label'

terma_detect_peaks(input_folder, output_folder)

Processing file: p000608-2167-03-09-11-54.csv
Peaks saved to: I:/dilated_cnn_peak_detection_model_data/train/train_data/label\p000608-2167-03-09-11-54_labeled_peaks.csv
Processing file: p000608-2167-03-23-08-12.csv
Peaks saved to: I:/dilated_cnn_peak_detection_model_data/train/train_data/label\p000608-2167-03-23-08-12_labeled_peaks.csv
Processing file: p000608-2167-03-24-19-50.csv
Peaks saved to: I:/dilated_cnn_peak_detection_model_data/train/train_data/label\p000608-2167-03-24-19-50_labeled_peaks.csv
Processing file: p000776-2184-04-30-15-16.csv
Peaks saved to: I:/dilated_cnn_peak_detection_model_data/train/train_data/label\p000776-2184-04-30-15-16_labeled_peaks.csv
Processing file: p000946-2120-05-14-08-08.csv
Peaks saved to: I:/dilated_cnn_peak_detection_model_data/train/train_data/label\p000946-2120-05-14-08-08_labeled_peaks.csv
Processing file: p004490-2151-01-07-12-36.csv
Peaks saved to: I:/dilated_cnn_peak_detection_model_data/train/train_data/label\p004490-2151-01-07-12-36_labe